### ShiTomashi Score

In [ ]:
%matplotlib inline
import cv2
import numpy as np
import random
import math
import matplotlib.pyplot as plt # matplotlibの描画系

In [ ]:
def cv_imshow(color):
    rgb = cv2.cvtColor(color, cv2.COLOR_BGR2RGB)
    plt.imshow(rgb)
    plt.show()

In [ ]:
lena = cv2.imread("data/lena/Lena.png")
lena = cv2.resize(lena, (256, 256))
cv_imshow(lena)
grey = cv2.cvtColor(lena, cv2.COLOR_BGR2GRAY)

探索ウィンドウ内の局所領域の勾配を$I_x$,$I_y$とする。それらから求まる以下の行列を

$$ 
\begin{eqnarray}
  M
  &=&
  \left(
    \begin{array}{c,c}
      d_{xx}& d_{xy} \\
      d_{xy}& d_{yy}
    \end{array}
  \right)
  &=&
  \sum_{x,y}
  \left(
    \begin{array}{c,c}
      I_xI_x & I_xI_y \\
      I_xI_y & I_yI_y
    \end{array}
  \right)  
\end{eqnarray}
$$ 
のように定義すると、ShiTomashi scoreは行列$M$の最小固有値となる。
$$
S_{ShiTomashi} = min(\lambda_1, \lambda_2)
$$
ちなみに、Harrisコーナーは
$$
S_{Harris} = \lambda_1\lambda_2 - k(\lambda_1 + \lambda_2)^2
$$

ここで、ShiTomashiは最小固有値だけを計算すれば良いので以下のように求まる。
$$
S_{ShiTomashi} = \frac{(d_{xx} + d_{yy})-\sqrt{(d_{xx} + d_{yy})^2-4(d_{xx}d_{yy} - d_{xy}^2})}{2}
$$
あとはソートして大きい値の特徴点を選ぶとか、しきい値以上の特徴点を選ぶということをして良い特徴点を得る。

In [ ]:
#detect features
detector = cv2.FastFeatureDetector_create()
detector.setThreshold(50)
detector.setNonmaxSuppression(True)
keyPoints = detector.detect(lena)

show_img = lena.copy()
for kp in keyPoints:
    pt = kp.pt
    cv2.circle(show_img, (int(pt[0]), int(pt[1])), 5, (0, 255, 0))
cv_imshow(show_img)

In [ ]:
def shiTomashiScore(image, x, y):
    u = int(x)
    v = int(y)
    half_box_size = 4
    box_size = 2 * half_box_size
    box_area = box_size * box_size 
    w = image.shape[0]
    h = image.shape[1]
    x_min = u - half_box_size
    x_max = u + half_box_size
    y_min = v - half_box_size
    y_max = v + half_box_size
    
    if(x_min < 1 or x_max > w - 1 or y_min < 1 or y_max > h - 1):
        return 0.0
    
    image_f = image.astype(np.float)
    
    top_p =  image_f[y_min - 1: y_max - 1, x_min: x_max]
    bottom_p = image_f[y_min + 1: y_max + 1, x_min: x_max]
    left_p =   image_f[y_min: y_max, x_min - 1: x_max - 1]
    right_p =image_f[y_min: y_max, x_min + 1: x_max + 1]
    dx_p = right_p - left_p
    dy_p = bottom_p - top_p
    dxx = (dx_p * dx_p).sum() / (2*box_area)
    dyy = (dy_p * dy_p).sum() / (2*box_area)
    dxy = (dx_p * dy_p).sum() / (2*box_area)
    B = dxx + dyy
    C = dxx * dyy - dxy * dxy
    return 0.5 * (B - math.sqrt(B * B - 4 * C))
    

In [ ]:
show_img = lena.copy()
score_list = []
for pt in keyPoints:
    score_list.append(shiTomashiScore(grey, pt.pt[0], pt.pt[1]))
index_list = np.argsort(np.array(score_list))[::-1]

for i in range(len(index_list)):
    if i > 30:
        break
    pt = keyPoints[index_list[i]].pt
    cv2.circle(show_img, (int(pt[0]), int(pt[1])), 5, (0, 255, 0))
cv_imshow(show_img)

 * Copyright (c) 2018, tmako123
 * https://twitter.com/tmako123
 * http://blog.livedoor.jp/tmako123-programming/
 * All rights reserved.
 * This file is distributed under the GNU Lesser General Public License v3.0.
 * The complete license agreement can be obtained at:
 * http://www.gnu.org/licenses/lgpl-3.0.html